# Fundamentals of Information Systems

## Python Programming (for Data Science)

### Master's Degree in Data Science

#### Gabriele Tolomei
<a href="mailto:gtolomei@math.unipd.it">gtolomei@math.unipd.it</a><br/>
University of Padua, Italy<br/>
2018/2019<br/>
January, 11 2019

# Lecture 13: The Classification Problem - Example (Part 2)

## Instructions

-  We consider the dataset file <code>**dataset.csv**</code>, which is contained in the <code>**loan-prediction**</code> directory on the Moodle page.

-  A description of the dataset is available in the <code>**README.txt**</code> file on the same directory.

-  **GOAL:** Use information from past loan applicants contained in <code>**dataset.csv**</code> to predict whether a _new_ applicant should be granted a loan or not.

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Import stats module from scipy, which contains a large number 
# of probability distributions as well as an exhaustive library of statistical functions.
import scipy.stats as stats

%matplotlib inline

## Summary of Part 1

In [2]:
# Path to the local dataset file
DATASET_PATH = "./data/loan-prediction/dataset.csv"

## Loading the Dataset

In [3]:
# Load the dataset with Pandas
data = pd.read_csv(DATASET_PATH, sep=",", index_col="Loan_ID")
print("Shape of the dataset: {}".format(data.shape))
data.head()
# NOTE: the first line of the file is considered as the header

Shape of the dataset: (614, 12)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


## Handling Missing Values (NA)

In [4]:
# is_numeric_dtype(pandas.Series) returns True iff the dtype associated with the pandas.Series is numeric
from pandas.api.types import is_numeric_dtype

data = data.apply(lambda x: x.fillna(x.median()) 
                      if is_numeric_dtype(x) 
                      else x.fillna(x.mode().iloc[0]))

## Handling Outliers

In [5]:
# Let's winsorize 'ApplicantIncome', 'CoapplicantIncome', and 'LoanAmount'
_ = stats.mstats.winsorize(data.ApplicantIncome, limits=0.05, inplace=True)
_ = stats.mstats.winsorize(data.CoapplicantIncome, limits=0.05, inplace=True)
_ = stats.mstats.winsorize(data.LoanAmount, limits=0.05, inplace=True)

# Apply log-transformation to 'ApplicantIncome' and assign it to a new column
data['Log_ApplicantIncome'] = data.ApplicantIncome.apply(np.log)
# Apply log-transformation to 'LoanAmount' and assign it to a new column
data['Log_LoanAmount'] = data.LoanAmount.apply(np.log)

## Encoding Categorical Features: One-Hot Encoding

In [6]:
# In pandas we can achieve easily one-hot encoding using the 'get_dummies()' function
categorical_features = [col for col in data.columns if not is_numeric_dtype(data[col]) and col != 'Loan_Status']
data_with_dummies = pd.get_dummies(data, columns = categorical_features)
data_with_dummies.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,Y,8.674026,4.852030,0,1,...,0,0,0,1,0,1,0,0,0,1
LP001003,4583,1508.0,128.0,360.0,1.0,N,8.430109,4.852030,0,1,...,1,0,0,1,0,1,0,1,0,0
LP001005,3000,0.0,66.0,360.0,1.0,Y,8.006368,4.189655,0,1,...,0,0,0,1,0,0,1,0,0,1
LP001006,2583,2358.0,120.0,360.0,1.0,Y,7.856707,4.787492,0,1,...,0,0,0,0,1,1,0,0,0,1
LP001008,6000,0.0,141.0,360.0,1.0,Y,8.699515,4.948760,0,1,...,0,0,0,1,0,1,0,0,0,1


In [7]:
# Just as a convention, I prefer to place the column to be predicted
# as the last one.
columns = data_with_dummies.columns.tolist()
# Popping out 'mpg' from the list and insert it back at the end.
columns.insert(len(columns), columns.pop(columns.index('Loan_Status')))
# Let's refactor the DataFrame using this new column index
data_with_dummies = data_with_dummies.loc[:, columns]
data_with_dummies.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,Married_No,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,8.674026,4.852030,0,1,1,...,0,0,1,0,1,0,0,0,1,Y
LP001003,4583,1508.0,128.0,360.0,1.0,8.430109,4.852030,0,1,0,...,0,0,1,0,1,0,1,0,0,N
LP001005,3000,0.0,66.0,360.0,1.0,8.006368,4.189655,0,1,0,...,0,0,1,0,0,1,0,0,1,Y
LP001006,2583,2358.0,120.0,360.0,1.0,7.856707,4.787492,0,1,0,...,0,0,0,1,1,0,0,0,1,Y
LP001008,6000,0.0,141.0,360.0,1.0,8.699515,4.948760,0,1,1,...,0,0,1,0,1,0,0,0,1,Y


## Encoding Binary Class Label

In [8]:
data = data_with_dummies
data.Loan_Status = data.Loan_Status.map(lambda x: 1 if x=='Y' else -1)
data.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,Married_No,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,8.674026,4.852030,0,1,1,...,0,0,1,0,1,0,0,0,1,1
LP001003,4583,1508.0,128.0,360.0,1.0,8.430109,4.852030,0,1,0,...,0,0,1,0,1,0,1,0,0,-1
LP001005,3000,0.0,66.0,360.0,1.0,8.006368,4.189655,0,1,0,...,0,0,1,0,0,1,0,0,1,1
LP001006,2583,2358.0,120.0,360.0,1.0,7.856707,4.787492,0,1,0,...,0,0,0,1,1,0,0,0,1,1
LP001008,6000,0.0,141.0,360.0,1.0,8.699515,4.948760,0,1,1,...,0,0,1,0,1,0,0,0,1,1


# 4. Building a Predictive Model

In [9]:
from sklearn.metrics import SCORERS
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

## 4.1 Splitting the Dataset: _Training_ vs. _Test_

In [10]:
"""
Extract the feature matrix from our original DataFrame.
"""
# Feature matrix X is composed of all the columns 
# except 'Loan_Status' (i.e., the target class label)
X = data.iloc[:,:-1]
X.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,Married_No,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,8.674026,4.852030,0,1,1,...,0,0,0,1,0,1,0,0,0,1
LP001003,4583,1508.0,128.0,360.0,1.0,8.430109,4.852030,0,1,0,...,1,0,0,1,0,1,0,1,0,0
LP001005,3000,0.0,66.0,360.0,1.0,8.006368,4.189655,0,1,0,...,0,0,0,1,0,0,1,0,0,1
LP001006,2583,2358.0,120.0,360.0,1.0,7.856707,4.787492,0,1,0,...,0,0,0,0,1,1,0,0,0,1
LP001008,6000,0.0,141.0,360.0,1.0,8.699515,4.948760,0,1,1,...,0,0,0,1,0,1,0,0,0,1


In [11]:
"""
Similarly, we want to extract the target class column vector y.
"""
y = data.Loan_Status
y.head()

Loan_ID
LP001002    1
LP001003   -1
LP001005    1
LP001006    1
LP001008    1
Name: Loan_Status, dtype: int64

In [ ]:
"""
Let's split our dataset with scikit-learn 'train_test_split' function, 
which splits the input dataset into training and test set, respectively.
We want the training set to account for 80% of the original dataset, whilst 
the test set to account for the remaining 20%.
Additionally, we would like to take advantage of stratified sampling,
so as to obtain the same target distribution in both the training and the test sets.
"""

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

In [13]:
print("Training Set shape: {}".format(X_train.shape))
print("Test Set shape: {}".format(X_test.shape))

Training Set shape: (491, 22)
Test Set shape: (123, 22)


## Feature Scaling: Why/When

-  <span style="color: red">**REMEMBER:**</span> Not every learning models are sensitive to different feature scales! 

-  For example, in the case of Logistic Regression the vector of model parameters we come up with when we minimize the negative log-likelihood - using gradient descent (iterative) solution - is **not** affected by different feature scales, except for a constant.

-  You can convince yourself of this by computing the gradient of the negative log-likelihood using non-scaled and scaled features.

-  Other models, instead, are not invariant with respect to scalar transformations of the input (features), and leads to completely different results if features are not properly scaled (e.g., Support Vector Machines or SVM). 

## Feature Scaling: How

-  Feature scaling **cannot** be done looking at the whole dataset!

-  In other words, either you standardize (using **z-scores**) or normalize (using **min-max**) your features you **must** do it considering only the training set portion of your dataset.

-  The same scaling, then, should be applied to the test set.

In [14]:
"""
Let's use two different feature scaling strategies: standard z-scores and min-max
"""
# The following is the scikit-learn package which provides
# various preprocessing capabilities
from sklearn import preprocessing

In [15]:
# Standardizing features using z-score
std_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_mean = X_train.mean()
# X_train_std = X_train.std()
# X_train_std = (X_train - X_train_mean)/X_train_std

# Normalizing features using min-max
minmax_scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_minmax = minmax_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_max = X_train.max()
# X_train_min = X_train.min()
# X_train_minmax = (X_train - X_train_min)/(X_train_max - X_train_min)

In [ ]:
"""
At this stage, we can work with 3 different feature matrices:
- The original one: X_train
- The standardized one: X_train_std
- The min-max normalized one: X_train_minmax

In the following, however, we work only on the original feature matrix X_train
"""

In [16]:
"""
General function used to assess the quality of predictions
in terms of two scores: accuracy and ROC AUC (Area Under the ROC Curve)
"""
def evaluate(true_values, predicted_values):
    # Classification Accuracy
    print("Accuracy = {:.3f}".
          format(accuracy_score(true_values, predicted_values)))
    # Explained variance score: 1 is perfect prediction
    print("Area Under the ROC Curve (ROC AUC) = {:.3f}".
          format(roc_auc_score(true_values, predicted_values)))

In [17]:
# Create logistic regression object
model = LogisticRegression()

# 1. Try to fit this logistic regressor to our original training set
model.fit(X_train, y_train)
# 2. Assess the quality of predictions made on the same training set
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print()
# 3. Assess the quality of predictions made on the test set
print("***** Evaluate Performance on Test Set *****") 
evaluate(y_test, model.predict(X_test))

***** Evaluate Performance on Training Set *****
Accuracy = 0.815
Area Under the ROC Curve (ROC AUC) = 0.717

***** Evaluate Performance on Test Set *****
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


In [18]:
print(classification_report(y_test, model.predict(X_test)))

             precision    recall  f1-score   support

         -1       0.89      0.45      0.60        38
          1       0.80      0.98      0.88        85

avg / total       0.83      0.81      0.79       123



In [19]:
# Simplest usage of cross-validation
model = LogisticRegression()
cv = cross_validate(model, X, y, cv=10, scoring=('roc_auc', 'accuracy'), return_train_score=True)
pd.DataFrame(cv)

,fit_time,score_time,test_accuracy,test_roc_auc,train_accuracy,train_roc_auc
0,0.004435,0.000981,0.793651,0.768605,0.811252,0.796818
1,0.004059,0.000849,0.825397,0.853488,0.807623,0.792554
2,0.004017,0.000875,0.786885,0.741855,0.811935,0.802723
3,0.004086,0.000837,0.754098,0.664160,0.817360,0.811120
4,0.004036,0.000886,0.770492,0.682957,0.813743,0.809598
5,0.003895,0.000856,0.786885,0.793233,0.811935,0.795330
6,0.003542,0.001171,0.868852,0.867168,0.802893,0.787466
7,0.005121,0.000929,0.836066,0.796992,0.806510,0.798494
8,0.004228,0.000860,0.803279,0.720551,0.808318,0.803575
9,0.002961,0.000828,0.836066,0.690476,0.806510,0.807286


In [20]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

***** Evaluate Average Performance on Training Set *****
Avg. Training Set Accuracy = 0.810
Avg. Training Set ROC AUC = 0.800

***** Evaluate Average Performance on Cross-Validation Set *****
Avg. Test Set Accuracy = 0.806
Avg. Test Set ROC AUC = 0.758


In [21]:
# Define an object of type KFold and pass it to the cross_validate function
model = LogisticRegression()
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)
cv = cross_validate(model, X, y, cv=k_fold, scoring=('roc_auc', 'accuracy'), return_train_score=True)
print(cv)

{'fit_time': array([ 0.00503278,  0.00406289,  0.00327516,  0.00363517,  0.00379109,
        0.0036242 ,  0.00361395,  0.00469089,  0.00469208,  0.00462389]), 'score_time': array([ 0.00176835,  0.00088501,  0.00088286,  0.00089478,  0.0008862 ,
        0.00083494,  0.00123739,  0.00116682,  0.00083804,  0.00088096]), 'test_roc_auc': array([ 0.80255517,  0.67670011,  0.825     ,  0.79559364,  0.69855072,
        0.78571429,  0.75062657,  0.72351421,  0.84404762,  0.77246377]), 'train_roc_auc': array([ 0.79542908,  0.8117478 ,  0.79228519,  0.79738283,  0.80807489,
        0.7958777 ,  0.80006085,  0.8064477 ,  0.79204556,  0.79751172]), 'test_accuracy': array([ 0.75806452,  0.80645161,  0.79032258,  0.83870968,  0.81967213,
        0.85245902,  0.81967213,  0.75409836,  0.80327869,  0.83606557]), 'train_accuracy': array([ 0.81521739,  0.8115942 ,  0.8115942 ,  0.80615942,  0.80831826,
        0.80831826,  0.80650995,  0.81555154,  0.80831826,  0.80650995])}


In [22]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

***** Evaluate Average Performance on Training Set *****
Avg. Training Set Accuracy = 0.810
Avg. Training Set ROC AUC = 0.800

***** Evaluate Average Performance on Cross-Validation Set *****
Avg. Test Set Accuracy = 0.808
Avg. Test Set ROC AUC = 0.767


In [23]:
# Define an object of type StratifiedKFold and pass it to the cross_validate function
model = LogisticRegression()
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=37)
cv = cross_validate(model, X, y, cv=k_fold, scoring=('roc_auc', 'accuracy'), return_train_score=True)
print(cv)

{'fit_time': array([ 0.00484776,  0.00375891,  0.00338387,  0.00368786,  0.00386786,
        0.00391293,  0.00358605,  0.00503516,  0.00514007,  0.00400209]), 'score_time': array([ 0.00086808,  0.00089002,  0.00089526,  0.0008688 ,  0.0008409 ,
        0.00087237,  0.000808  ,  0.00119281,  0.00125813,  0.00128603]), 'test_roc_auc': array([ 0.76976744,  0.79069767,  0.76190476,  0.81829574,  0.81077694,
        0.72305764,  0.6641604 ,  0.72556391,  0.82706767,  0.70551378]), 'train_roc_auc': array([ 0.80097564,  0.79694115,  0.79776392,  0.79370246,  0.79522361,
        0.80629754,  0.80800122,  0.80219045,  0.79177061,  0.80915729]), 'test_accuracy': array([ 0.79365079,  0.87301587,  0.83606557,  0.85245902,  0.75409836,
        0.7704918 ,  0.78688525,  0.78688525,  0.81967213,  0.78688525]), 'train_accuracy': array([ 0.81125227,  0.80036298,  0.80831826,  0.80650995,  0.81555154,
        0.8119349 ,  0.81374322,  0.81012658,  0.80831826,  0.81374322])}


In [24]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

***** Evaluate Average Performance on Training Set *****
Avg. Training Set Accuracy = 0.810
Avg. Training Set ROC AUC = 0.800

***** Evaluate Average Performance on Cross-Validation Set *****
Avg. Test Set Accuracy = 0.806
Avg. Test Set ROC AUC = 0.760


## Model Selection and Evaluation

-  So far, we have just focused on a very specific _instance_ of a Logistic Regression model.

-  In other words, we haven't spent time trying to _tune_ any "meta-parameter" (known as **hyperparameter**) of our model.

-  In practice, we used default values of hyperparameters for our Logistic Regression model, according to <code>**scikit-learn**</code>

-  We didn't perform any actual model selection, as hyperparameters are fixed!

-  The figures we output for test accuracy/ROC AUC scores are our estimates of _generalization_ performance of our model (i.e., evaluation)

## Model Selection and Evaluation (cont'd)

-  Most of the time, though, we may need to do one of the following:
    1.  Fix a "family" of models (e.g., Logistic Regression) and perform hyperparameter selection;
    2.  Choose between a set of models (e.g., Logistic Regression, SVM, Decision Tree, etc.), each one with a fixed (i.e., default) set of hyperparameters;
    3.  A mixture of the above, where we have to select the best hyperparameters of the best model picked from a set of different models.

-  In any case, we also need to provide an estimate of the generalization performance of the chosen model.

## Case 1: Select Best Hyperparameters of a Fixed Family of Models

## 1.1: Using Validation Set

In [25]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2]}
                                                )
                         }

In [26]:
# Outer splitting: Training vs. Test set (e.g., 80÷20) used to separate training-selection-evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

# Inner splitting (i.e., within the outer training set): Training vs. Validation (e.g., 80÷20)
# Training set is used to train the model, validation set is used to select the best hyperparameters
X_train_train, X_validation, y_train_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y_train)

In [27]:
# Keep the training score obtained with each hyperparameter
training_scores = {}
# Keep the validation score obtained with each hyperparameter
validation_scores = {}
# Keep only the best training/validation scores
best_training_score = {}
best_validation_score = {}

# Get the only model available
model = models_and_hyperparams['LogisticRegression'][0]
# Get the hyperparameters for that model
hyperparams = models_and_hyperparams['LogisticRegression'][1]

# Loop through all the hyperparameters
for hp in hyperparams:
    training_scores[hp] = {}
    validation_scores[hp] = {}
    # Loop through all the value of a specific hyperparameter
    for val in hyperparams[hp]:
        # set the model's hyperparameter to the current value
        model.set_params(**{hp: val})
        # fit the model on the inner training portion 
        model.fit(X_train_train, y_train_train)
        # store the inner training score
        training_score = accuracy_score(y_train_train, model.predict(X_train_train))
        training_scores[hp][val] = training_score
        # store the inner validation score
        validation_score = accuracy_score(y_validation, model.predict(X_validation))
        validation_scores[hp][val] = validation_score
        # Update best training/validation scores
        if not best_training_score:
            best_training_score[hp] = (val, training_score)
        else:
            if best_training_score[hp][1] < training_score:
                best_training_score[hp] = (val, training_score)
                
        if not best_validation_score:
            best_validation_score[hp] = (val, validation_score)
        else:
            if best_validation_score[hp][1] < validation_score:
                best_validation_score[hp] = (val, validation_score)

In [28]:
print("***** Evaluate Performance on Training Set *****")
print(training_scores)
print("***** Evaluate Performance on Validation Set *****")
print(validation_scores)
print("***** Best Accuracy Score on Training Set *****")
print(best_training_score)
print("***** Best Accuracy Score on Validation Set *****")
print(best_validation_score)

***** Evaluate Performance on Training Set *****
{'C': {0.01: 0.68622448979591832, 0.05: 0.78061224489795922, 0.1: 0.81122448979591832, 0.5: 0.81887755102040816, 1: 0.81632653061224492, 2: 0.81632653061224492}}
***** Evaluate Performance on Validation Set *****
{'C': {0.01: 0.68686868686868685, 0.05: 0.75757575757575757, 0.1: 0.76767676767676762, 0.5: 0.79797979797979801, 1: 0.79797979797979801, 2: 0.78787878787878785}}
***** Best Accuracy Score on Training Set *****
{'C': (0.5, 0.81887755102040816)}
***** Best Accuracy Score on Validation Set *****
{'C': (0.5, 0.79797979797979801)}


In [29]:
# We set the model's hyperparameters to those leading to the best score on the validation test
best_params = dict([(list(best_validation_score.keys())[0], list(best_validation_score.values())[0][0])])
model.set_params(**best_params)
# We fit this model to the whole training set portion
model.fit(X_train, y_train)
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print("***** Evaluate Performance on Test Set *****")
evaluate(y_test, model.predict(X_test))

***** Evaluate Performance on Training Set *****
Accuracy = 0.815
Area Under the ROC Curve (ROC AUC) = 0.713
***** Evaluate Performance on Test Set *****
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


## 1.2.a: Using Cross-Validation (Single Hyperparameter)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

In [31]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2]}
                                                )
                         }

In [32]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# Get the only model available
model = models_and_hyperparams['LogisticRegression'][0]
# Get the hyperparameters for that model
hyperparams = models_and_hyperparams['LogisticRegression'][1]

gs = GridSearchCV(estimator=model, param_grid=hyperparams, cv=k_fold, 
                  scoring='accuracy',
                  verbose=True,
                 return_train_score=True)
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.2s finished


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split7_test_score,split7_train_score,split8_test_score,split8_train_score,split9_test_score,split9_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.002772,0.000375,0.684318,0.689973,0.01,{'C': 0.01},6,0.68,0.691610,0.68,...,0.687500,0.697517,0.687500,0.688488,0.666667,0.686230,0.000352,0.000047,0.008209,0.003449
1,0.003276,0.000413,0.763747,0.777785,0.05,{'C': 0.05},5,0.76,0.782313,0.74,...,0.750000,0.767494,0.770833,0.769752,0.729167,0.783296,0.000488,0.000152,0.027821,0.007463
2,0.002709,0.000271,0.802444,0.806290,0.1,{'C': 0.1},4,0.80,0.811791,0.74,...,0.791667,0.808126,0.791667,0.801354,0.750000,0.814898,0.000268,0.000046,0.035839,0.005934
3,0.002787,0.000261,0.810591,0.815339,0.5,{'C': 0.5},1,0.80,0.820862,0.76,...,0.791667,0.817156,0.833333,0.812641,0.750000,0.828442,0.000212,0.000058,0.035779,0.006581
4,0.002707,0.000246,0.810591,0.815114,1,{'C': 1},1,0.80,0.818594,0.76,...,0.791667,0.814898,0.833333,0.810384,0.750000,0.828442,0.000094,0.000013,0.039045,0.005516
5,0.003345,0.000336,0.806517,0.814889,2,{'C': 2},3,0.78,0.823129,0.76,...,0.791667,0.814898,0.833333,0.810384,0.750000,0.826185,0.000708,0.000118,0.036740,0.006492


In [33]:
print("Best hyperparameter: {}".format(gs.best_params_))
print("Best accuracy score: {:.3f}".format(gs.best_score_))
evaluate(y_test, gs.predict(X_test))

Best hyperparameter: {'C': 0.5}
Best accuracy score: 0.811
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


## 1.2.b: Using Cross-Validation (Multiple Hyperparameters)

In [34]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2],
                                                 'penalty': ['l1', 'l2']}
                                                )
                         }

In [35]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=31)
# Get the only model available
model = models_and_hyperparams['LogisticRegression'][0]
# Get the hyperparameters for that model
hyperparams = models_and_hyperparams['LogisticRegression'][1]

gs = GridSearchCV(estimator=model, param_grid=hyperparams, cv=k_fold, 
                  scoring='accuracy',
                  verbose=True,
                 return_train_score=True)
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    0.6s finished


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_penalty,params,rank_test_score,split0_test_score,split0_train_score,...,split7_test_score,split7_train_score,split8_test_score,split8_train_score,split9_test_score,split9_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.001959,0.000368,0.682281,0.688393,0.01,l1,"{'C': 0.01, 'penalty': 'l1'}",12,0.68,0.691610,...,0.666667,0.681716,0.687500,0.688488,0.666667,0.690745,0.000222,0.000087,0.013198,0.002967
1,0.003046,0.000374,0.684318,0.688846,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",11,0.68,0.689342,...,0.687500,0.688488,0.687500,0.683973,0.687500,0.690745,0.000709,0.000094,0.009810,0.002725
2,0.001817,0.000259,0.808554,0.808330,0.05,l1,"{'C': 0.05, 'penalty': 'l1'}",1,0.74,0.816327,...,0.833333,0.805869,0.791667,0.810384,0.833333,0.805869,0.000328,0.000032,0.031273,0.003493
3,0.002625,0.000259,0.771894,0.777102,0.05,l2,"{'C': 0.05, 'penalty': 'l2'}",10,0.70,0.791383,...,0.791667,0.774266,0.791667,0.778781,0.812500,0.776524,0.000207,0.000042,0.035235,0.005506
4,0.001996,0.000241,0.806517,0.808557,0.1,l1,"{'C': 0.1, 'penalty': 'l1'}",4,0.74,0.816327,...,0.833333,0.805869,0.791667,0.810384,0.833333,0.805869,0.000292,0.000010,0.032395,0.003490
5,0.002634,0.000256,0.802444,0.806975,0.1,l2,"{'C': 0.1, 'penalty': 'l2'}",9,0.74,0.814059,...,0.833333,0.803612,0.791667,0.803612,0.833333,0.805869,0.000191,0.000049,0.040758,0.004174
6,0.004161,0.000334,0.808554,0.814215,0.5,l1,"{'C': 0.5, 'penalty': 'l1'}",1,0.76,0.818594,...,0.833333,0.812641,0.791667,0.812641,0.833333,0.812641,0.000562,0.000068,0.034808,0.003183
7,0.002764,0.000248,0.806517,0.815572,0.5,l2,"{'C': 0.5, 'penalty': 'l2'}",4,0.76,0.823129,...,0.833333,0.817156,0.791667,0.812641,0.791667,0.817156,0.000160,0.000013,0.035534,0.004072
8,0.004050,0.000241,0.808554,0.815344,1,l1,"{'C': 1, 'penalty': 'l1'}",1,0.76,0.820862,...,0.833333,0.812641,0.791667,0.817156,0.812500,0.819413,0.000800,0.000006,0.035220,0.003740
9,0.002786,0.000262,0.804481,0.815799,1,l2,"{'C': 1, 'penalty': 'l2'}",7,0.76,0.820862,...,0.833333,0.817156,0.791667,0.812641,0.791667,0.817156,0.000200,0.000052,0.034101,0.003915


In [36]:
print("Best hyperparameter: {}".format(gs.best_params_))
print("Best accuracy score: {:.3f}".format(gs.best_score_))
evaluate(y_test, gs.predict(X_test))

Best hyperparameter: {'C': 0.05, 'penalty': 'l1'}
Best accuracy score: 0.809
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


## Case 2: Select Best Model out of a Set of Family of Models with Fixed Hyperparameters

## Using Cross Validation

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

In [38]:
models = {'LogisticRegression': LogisticRegression(),
          'LinearSVC': LinearSVC(),
          'DecisionTreeClassifier': DecisionTreeClassifier(),
          'RandomForestClassifier': RandomForestClassifier(),
          'GradientBoostingClassifier': GradientBoostingClassifier()
          # Add other families of models here...
         }

In [39]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = {}
for model_name, model in models.items():
    cv_scores[model_name] = cross_val_score(model, X_train, y_train, cv=k_fold, scoring='accuracy')

In [40]:
cv_df = pd.DataFrame(cv_scores).transpose()
cv_df

,0,1,2,3,4,5,6,7,8,9
DecisionTreeClassifier,0.68,0.68,0.58,0.64,0.734694,0.693878,0.673469,0.645833,0.645833,0.666667
GradientBoostingClassifier,0.76,0.72,0.72,0.76,0.857143,0.816327,0.755102,0.833333,0.812500,0.687500
LinearSVC,0.68,0.32,0.68,0.68,0.367347,0.693878,0.693878,0.312500,0.687500,0.687500
LogisticRegression,0.80,0.76,0.80,0.88,0.836735,0.857143,0.795918,0.791667,0.833333,0.750000
RandomForestClassifier,0.76,0.64,0.78,0.78,0.693878,0.734694,0.693878,0.729167,0.750000,0.770833


In [41]:
cv_df['avg_cv'] = np.mean(cv_df, axis=1)
cv_df['std_cv'] = np.std(cv_df, axis=1)
cv_df = cv_df.sort_values(['avg_cv', 'std_cv'], ascending=[False,True])
cv_df

,0,1,2,3,4,5,6,7,8,9,avg_cv,std_cv
LogisticRegression,0.80,0.76,0.80,0.88,0.836735,0.857143,0.795918,0.791667,0.833333,0.750000,0.810480,0.037190
GradientBoostingClassifier,0.76,0.72,0.72,0.76,0.857143,0.816327,0.755102,0.833333,0.812500,0.687500,0.772190,0.050216
RandomForestClassifier,0.76,0.64,0.78,0.78,0.693878,0.734694,0.693878,0.729167,0.750000,0.770833,0.733245,0.041115
DecisionTreeClassifier,0.68,0.68,0.58,0.64,0.734694,0.693878,0.673469,0.645833,0.645833,0.666667,0.664037,0.036669
LinearSVC,0.68,0.32,0.68,0.68,0.367347,0.693878,0.693878,0.312500,0.687500,0.687500,0.580260,0.154751


In [42]:
# Model Selection: Logistic Regression is the best overall method, therefore we pick that!
# Now we need to provide an estimate of its generalization performance. 
# To do so, we evaluate it against the test set portion we previously held out.
model = models[cv_df.index[0]]
# Re-fit the best selected model on the whole training set
model.fit(X_train, y_train)
# Evaluation
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print("***** Evaluate Performance on Test Set *****")
evaluate(y_test, model.predict(X_test))

***** Evaluate Performance on Training Set *****
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.714
***** Evaluate Performance on Test Set *****
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


## Case 3: Select the Best Hyperparameters AND the Best Model from a Family of Models

In [ ]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2],
                                                 'penalty': ['l1', 'l2']}
                                                ),
                          'RandomForestClassifier': (RandomForestClassifier(),
                                       {'n_estimators': [10, 50, 100]}
                                       ),
                          'DecisionTreeClassifier': (DecisionTreeClassifier(),
                                                     {'criterion': ['gini', 'entropy'], 
                                                      'max_depth': [i for i in range(1, X.shape[1]+1)]}
                                                    )
                         }

In [ ]:
# `outer_cv` creates 10 folds for estimating generalization error
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=73)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=37, 
                                                    stratify=y)

# we will collect the average of the scores on the 10 outer folds in this dictionary
# with keys given by the names of the models in `models_and_hyperparams`
average_scores_across_outer_folds_for_each_model = dict()

# find the model with the best generalization error
for name, (model, params) in models_and_hyperparams.items():
    # this object is a classifier that also happens to choose
    # its hyperparameters automatically using `inner_cv`
    model_optimizing_hyperparams = GridSearchCV(estimator=model, 
                                                param_grid=params,
                                                cv=inner_cv, 
                                                scoring='accuracy',
                                               verbose=True)

    # estimate generalization error on the 10-fold splits of the data
    scores_across_outer_folds = cross_val_score(model_optimizing_hyperparams,
                                                X_train, y_train, cv=outer_cv, scoring='accuracy')

    # get the mean accuracy across each of outer_cv's 10 folds
    average_scores_across_outer_folds_for_each_model[name] = np.mean(scores_across_outer_folds)
    performance_summary = 'Model: {name}\nAccuracy in the 10 outer folds: {scores}.\nAverage Accuracy: {avg}'
    print(performance_summary.format(
        name=name, scores=scores_across_outer_folds,
        avg=np.mean(scores_across_outer_folds)))
    print()

print('Average score across the outer folds: ',
      average_scores_across_outer_folds_for_each_model)

many_stars = '\n' + '*' * 100 + '\n'
print(many_stars + 'Now we choose the best model and refit on the whole dataset' + many_stars)

best_model_name, best_model_avg_score = max(
    average_scores_across_outer_folds_for_each_model.items(),
    key=(lambda name_averagescore: name_averagescore[1]))

# get the best model and its associated parameter grid
best_model, best_model_params = models_and_hyperparams[best_model_name]

# now we refit this best model on the whole dataset so that we can start
# making predictions on other data, and now we have a reliable estimate of
# this model's generalization error and we are confident this is the best model
# among the ones we have tried
final_model = GridSearchCV(best_model, best_model_params, cv=inner_cv)
final_model.fit(X_train, y_train)

print('Best model: \n\t{}'.format(best_model), end='\n\n')
print('Estimation of its generalization performance (accuracy):\n\t{}'.format(
    best_model_avg_score), end='\n\n')
print('Best parameter choice for this model: \n\t{params}'
      '\n(according to cross-validation `{cv}` on the whole dataset).'.format(
      params=final_model.best_params_, cv=inner_cv))


y_true, y_pred, y_pred_prob = y, final_model.predict(X), final_model.predict_proba(X)
print()
print(classification_report(y_true, y_pred))
roc = roc_auc_score(y_true, y_pred_prob[:,1])
acc = accuracy_score(y_true, y_pred)
print("Accuracy = [{:.3f}]".format(acc))
print("Area Under the ROC = [{:.3f}]".format(roc))